# Computer Vision Analysis
#### By Ronny Toribio, Kadir O. Altunel, Michael Cook-Stahl
#### Based on [Hands on Machine Learning 2nd edition](https://github.com/ageron/handson-ml2/) and [FER2013](https://www.kaggle.com/code/ritikjain00/model-training-fer-13)

### Import modules

In [ ]:
%matplotlib inline
import os.path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

### Load Facial Emotion Recognition dataset

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, horizontal_flip=True)
train_X, train_y = train_datagen.flow_from_directory("archive/train", batch_size=64,
                                          target_size=(48, 48), shuffle=True,
                                          color_mode="grayscale", class_mode="categorical")[0]

test_datagen = ImageDataGenerator(rescale=1./255)
test_X, test_y = test_datagen.flow_from_directory("archive/test", batch_size=64,
                                       target_size=(48, 48), shuffle=True,
                                       color_mode="grayscale", class_mode="categorical")[0]

### Build CNN model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_size, padding="same", activation="relu"),
    Conv2D(64, (3, 3), padding="same", activation="relu"),
    BatchNormalization()
    MaxPool2D((2, 2), strides=(2, 2), padding="valid"),
    Dropout(0.25),
    Conv2D(128, (3, 3), padding="same", activation="relu", kernel_regularization=regularizers.l2(0.01)),
    Conv2D(256, (3, 3), padding="same", activation="relu", kernel_regularization=regularizers.l2(0.01)),
    BatchNormalization()
    MaxPool2D((2, 2), strides=(2, 2), padding="valid"),
    Dropout(0.25),
    Flatten(),
    Dense(1024, activation="relu"),
    Dropout(0.5)
    Dense(7, activation="softmax")
])

### Model properties

In [ ]:
model.layers

In [ ]:
model.summary()

In [ ]:
plot_model(model, "cnn_model.png", show_shapes=True)

### Compile model

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Model checkpoint callback

In [ ]:
checkpoint_cb = ModelCheckpoint("cnn_model.h5", save_best_only=True)

### Train the model

In [ ]:
history = model.fit(X_train, y_train, epoch=30, validation_data=(X_test, y_test), callbacks=[checkpoint_cb])

### Roll back to the best model

In [ ]:
model = load_model("cnn_model.h5")

### Plot training history

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
save_fig("cnn_model_training.png")
plt.show()

### Evaluate model

In [ ]:
model.evaluate(X_test, y_test)

### Save weights

In [ ]:
model.save_weights("cnn_model_weights.ckpt")